In [6]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import hour, weekofyear, date_format,dayofweek
from pyspark.sql.functions import year,month, dayofmonth
from pyspark.sql.functions import monotonically_increasing_id
#from pyspark.sql.functions import to_timestamp
from pyspark.sql.types import StructType, StructField, DoubleType, StringType
from pyspark.sql.types import IntegerType, DateType

In [32]:
from pyspark.sql.functions import split, explode

In [75]:
from pyspark.sql.functions import col, concat_ws

In [7]:

def create_spark_session():
    """Creates SparkSession. General configurarion
    of the script.
    """
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

In [8]:
spark = create_spark_session()

In [10]:
dataPath = "./arxiv-metadata-oai-snapshot.json"
df = spark.read.json(dataPath)

In [11]:
print(df.count())

1753042


In [12]:
df.printSchema()

root
 |-- abstract: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- authors_parsed: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- categories: string (nullable = true)
 |-- comments: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- id: string (nullable = true)
 |-- journal-ref: string (nullable = true)
 |-- license: string (nullable = true)
 |-- report-no: string (nullable = true)
 |-- submitter: string (nullable = true)
 |-- title: string (nullable = true)
 |-- update_date: string (nullable = true)
 |-- versions: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- created: string (nullable = true)
 |    |    |-- version: string (nullable = true)



In [86]:
df_autor=df.selectExpr("id","explode(authors_parsed) as e").withColumn("new_au",concat_ws(" ",col("e")))

In [88]:
df_autor.select("id","new_au").show(1)

+---------+----------+
|       id|    new_au|
+---------+----------+
|0704.0001|Balázs C. |
+---------+----------+
only showing top 1 row



In [ ]:
df2 = df.withColumn("languagesAtSchool",
   concat_ws(",",col("languagesAtSchool")))

In [45]:
df.select("authors").limit(10).toPandas()

,authors
0,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan"
1,Ileana Streinu and Louis Theran
2,Hongjun Pan
3,David Callan
4,Wael Abu-Shammala and Alberto Torchinsky
5,Y. H. Pong and C. K. Law
6,"Alejandro Corichi, Tatjana Vukasinac and Jose A. Zapata"
7,Damian C. Swift
8,"Paul Harvey, Bruno Merin, Tracy L. Huard, Luisa M. Rebull, Nicholas\n Chapman, Neal J. Evans II, Philip C. Myers"
9,Sergei Ovchinnikov


In [39]:
df.show(2)

+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+---------+--------------------+--------------------+----------------+--------------+--------------------+-----------+--------------------+
|            abstract|             authors|      authors_parsed|   categories|            comments|                 doi|       id|         journal-ref|             license|       report-no|     submitter|               title|update_date|            versions|
+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+---------+--------------------+--------------------+----------------+--------------+--------------------+-----------+--------------------+
|  A fully differe...|C. Bal\'azs, E. L...|[[Balázs, C., ], ...|       hep-ph|37 pages, 15 figu...|10.1103/PhysRevD....|0704.0001|Phys.Rev.D76:0130...|                null|ANL-HEP-PR-07-12|Pavel Nadolsky|Calculation of pr..

In [ ]:
df.selectExpr("explode(somethingelse.mySomething.arr) as e").selectExpr("explode(e.elem) as e")